In [30]:
import random
import math
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from functools import partial
import statistics


In [31]:
def rw2(n):# this gives samples from weibull(2,5)
    x,y=0,0
    distance=[]
    for i in range(1,n+1):
            r=random.weibullvariate(alpha,beta) # step size r
            theta=2.*math.pi*random.random()
            x +=r*math.cos(theta)
            y +=r*math.sin(theta)
            distance.append(r)# save the distances r for the mcmc sampler
    return(np.array(distance))# depending what do you want to do, either save the pairs, or save the distances r  between points
    #return(x,y)
    
alpha=int(input("what's the scale?"))
beta=int(input("what's the shape?"))
n=int(input("how many steps?"))
tries=int(input("how many tries?"))#  put tries=1  in order to properly save the list distance
for i in range(tries):
    print (rw2(n))

what's the scale?2
what's the shape?5
how many steps?100
how many tries?1
[ 2.28029695  0.8739349   1.57438205  2.02994377  0.63730457  1.67131642
  2.09195888  1.32265306  1.10090816  2.4091576   1.97725891  1.71084839
  2.01429403  2.52081825  1.85965743  1.76685     2.09699618  1.33390349
  1.58320886  2.4240859   1.98640584  2.19643505  1.72459628  0.85435632
  1.22170542  1.81580095  1.33127185  1.69825571  0.93228528  2.36525215
  1.30504972  1.5276808   2.05896841  1.63462707  0.65609302  1.88999076
  1.92743567  1.52986614  1.6448298   1.14376846  2.05333084  1.71575823
  1.25828976  1.53096712  1.78864742  1.79731298  2.41724605  1.82345497
  2.09793252  2.23241782  2.08076607  1.41365195  1.40894517  1.99272386
  2.02135988  1.90011021  1.60901367  1.5898703   1.76776373  2.07107661
  2.01915636  2.26857161  2.20855841  2.48016432  1.69471845  2.81198195
  2.09018894  1.86440709  1.90833199  0.56174126  2.07329686  1.35295375
  2.14710338  2.18186282  1.58087912  1.70836122  

In [32]:
# calculate the log likelihood of weibull(a,b)
import scipy.stats 
from scipy.stats import dweibull
def log_likelihood(x,a,b):
     return sum(dweibull.logpdf(x,c=a,scale=b))
    

In [52]:
n=100 #number of observations in a sample 
# initial guess for theta(a and b) as array.
guess = [2.0,5.0]
# Prepare storing MCMC chain as array of arrays.
A = [guess]
# define stepsize of MCMC.
stepsizes = [0.005,0.005]  # array of stepsizes
accepted  = 0.0
old_theta=guess# define initial values for theta 
old_loglik = log_likelihood(rw2(n),old_theta[0],old_theta[1])#calculate the first log likelihood
# Metropolis-Hastings with 10,000 iterations.
for p in range(10000):
    #old_theta  = A[len(A)-1]   old parameter value as array(not that elegant way)
    # Suggest new candidate from Gaussian proposal distribution.
    #new_theta = np.zeros([len(old_theta)])
    new_theta = old_theta + stats.norm(0, stepsizes).rvs() 
    #add the restraints on new_theta
    if new_theta[0]<0 or new_theta[1]<0: 
        continue
    new_loglik = log_likelihood(rw2(n),new_theta[0],new_theta[1])
    # Accept new candidate in Monte-Carlo fashing.
    if (new_loglik > old_loglik):
        A.append(new_theta)
        accepted = accepted + 1.0  # monitor acceptance
        old_loglik=new_loglik
        old_theta=new_theta
    else:
        u = random.uniform(0.0,1.0)
        if (u < math.exp(new_loglik - old_loglik)):
            A.append(new_theta)
            accepted = accepted + 1.0  # monitor acceptance
            old_loglik=new_loglik
            old_theta=new_theta
        #else:
            #A.append(old_alpha)(I guess this is not good)
print("Acceptance rate = "+str(accepted/10000.0))
#print(A)



Acceptance rate = 0.5473


In [16]:
old_theta=guess# define initial values for theta 
old_loglik = log_likelihood(rw2(n),old_theta[0],old_theta[1])#calculate the first log likelihood

In [20]:
new_theta = np.zeros([len(old_theta)])
new_theta

array([ 0.,  0.])

In [28]:
# I believe the two methods are the same
import scipy.stats as stats
old_theta=[2,5]
stepsizes=[0.005,0.005]
new_theta=np.zeros(2)
new_theta[0] = random.gauss(old_theta[0], stepsizes[0])
new_theta[1] = random.gauss(old_theta[1], stepsizes[1])
print(new_theta)
#?random.gauss
sigma=0.3
theta_p = old_theta + stats.norm(0, sigma).rvs() 
print(theta_p)
stats.norm(0, sigma).rvs() 
#?stats.norm
#newest_theta=random.gauss(old_theta,stepsizes)doesn't work
#newest_theta
newest_theta=stats.norm(old_theta,sigma).rvs()
print(newest_theta)

[ 1.99873478  4.9989597 ]
[ 1.78351004  4.78351004]
[ 1.87770982  4.84593636]


In [68]:
# save the posterior for a
a=[]
for i in range(len(A)-1):
    a.append(A[i-1][0])
#print(a,end='')   
statistics.mean(a)   

1.7563765015009953

In [67]:
# save the posterior for b
b=[]
for i in range(len(A)-1):
    b.append(A[i-1][1])
#print(b,end='') 
statistics.mean(b)

4.0117643876478564

In [46]:
okay_theta = old_theta + stats.norm(0, stepsizes).rvs() 
okay_theta

array([ 1.89933029,  2.54323802])